In [ ]:
"""下载代码
"""
%cd /content/
!git clone https://github.com/billvsme/my_finetune

In [ ]:
"""下载chatglm3-base模型, 时间较长请耐心等待
"""
%cd /content/
!git clone --depth=1 https://huggingface.co/THUDM/chatglm3-6b-base

In [ ]:
"""安装环境
"""
%cd /content/my_finetune

!apt install -r python3.10-venv
!mkdir ~/.venv
!python -m venv ~/.venv/finetune
!~/.venv/finetune/bin/pip install -r requirements.txt

In [ ]:
"""替换自我认知self_cognition数据集中的名称
"""
%cd /content/my_finetune
!sed -i 's/<NAME>/法律AI/g' data/self_cognition.json
!sed -i 's/<AUTHOR>/billvsme/g' data/self_cognition.json

In [ ]:
"""生成deepspeed配置文件
"""
%cd /content/my_finetune
!echo '''{\
  "train_batch_size": "auto",\
  "train_micro_batch_size_per_gpu": "auto",\
  "gradient_accumulation_steps": "auto",\
  "gradient_clipping": "auto",\
  "zero_allow_untested_optimizer": true,\
  "fp16": {\
    "enabled": "auto",\
    "loss_scale": 0,\
    "initial_scale_power": 16,\
    "loss_scale_window": 1000,\
    "hysteresis": 2,\
    "min_loss_scale": 1\
  },\
  "zero_optimization": {\
    "stage": 2,\
    "allgather_partitions": true,\
    "allgather_bucket_size": 1e8,\
    "reduce_scatter": true,\
    "reduce_bucket_size": 1e8,\
    "overlap_comm": true,\
    "contiguous_gradients": true\
  }\
}''' > ds_config.json

In [ ]:
"""进行qlora sft微调 🤩
"""
%cd /content/my_finetune
!~/.venv/finetune/bin/deepspeed --num_gpus 1 --master_port=9901 finetune.py \
    --deepspeed ds_config.json \
    --model_name_or_path /content/chatglm3-6b-base \
    --do_train True\
    --data_dir /content/my_finetune/data/ \
    --data_filename self_cognition.json  \
    --max_source_length 2048 \
    --max_target_length 2048 \
    --max_samples 80 \
    --quantization_bit 4 \
    --learning_rate 2e-4 \
    --lr_scheduler_type linear \
    --max_grad_norm 0.5 \
    --adam_beta1 0.9 \
    --adam_beta2 0.999 \
    --adam_epsilon 1e-8 \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.1 \
    --preprocessing_num_workers 4 \
    --per_device_train_batch_size 16 \
    --gradient_accumulation_steps 1 \
    --max_steps 100 \
    --logging_steps 1 \
    --save_steps 1000 \
    --output_dir output/chatglmt3_qlora \
    --overwrite_output_dir True \
    --fp16 True

In [ ]:
"""查看微调结果😁
"""
%cd /content/my_finetune
!~/.venv/finetune/bin/python inference.py \
    --model_name_or_path /content/chatglm3-6b-base \
    --lora_path output/chatglmt3_qlora \
    --data_dir /content/my_finetune/data/ \
    --data_filename self_cognition.json\
    --max_samples 80 \
    --quantization_bit 4
